In [57]:
import os
import xml.etree.cElementTree as ET
from lxml import etree
import cv2


def yolo_to_cor(centerX, centerY, width, height, resolX, resolY):
    x1 = (centerX - (width / 2)) * resolX
    y1 = (centerY -(height / 2)) * resolY
    x2 = (centerX + (width / 2)) * resolX
    y2 = (centerY +(height / 2)) * resolY
    
    return x1, y1, x2, y2
    

def text_to_xml(folder_path, file):   
    
    filename = file.split('.')[0] # 파일명
    extention = file.split('.')[1] # 확장자명
      
    
    image = cv2.imread(os.path.join(folder_path, file))
    height, width, depth = image.shape
    
    file_full_path = os.path.join(folder_path, filename) + '.txt' # 경로폴더와 파일이름을 합친다
   
    save_path = os.path.join(folder_path, "{}.xml".format(filename))
    
  
    
    
    
    try:        
        f = open(file_full_path, 'r')
        
        annotation = ET.Element('annotation')
        ET.SubElement(annotation, 'folder').text = "train"
        ET.SubElement(annotation, 'filename').text = filename + '.jpg'
        ET.SubElement(annotation, 'path').text = os.path.join(folder_path, file)

        source = ET.SubElement(annotation, 'source')
        ET.SubElement(source, 'database').text = "Unknown"    

        size = ET.SubElement(annotation, 'size')
        ET.SubElement(size, 'width').text = str(width)
        ET.SubElement(size, 'height').text = str(height)
        ET.SubElement(size, 'depth').text = str(depth)

        ET.SubElement(annotation, 'segmented').text = '0'
        
        for line in f:
            content = line.split(' ') # 구분자
        
            
            if content[0] == '0': 
                classes = 'head'
            elif content[0] == '2': 
                classes = 'person'
            else:
                classes = 'helmet'
                
            #print(content[0], classes)

            #xmin, xmax, ymin, ymax = content[1], content[2], content[3], content[4]
            # yolo에 맞춰 실수중심점과 w,h로 작성된 데이터를 Cnn에 맞게 정수x1x2y1y2로 변환
            x1, y1, x2, y2 = yolo_to_cor(float(content[1]),float(content[2]),float(content[3]),float(content[4]), width, height)
            
            ob = ET.SubElement(annotation, 'object')
            ET.SubElement(ob, 'name').text = classes
            ET.SubElement(ob, 'pose').text = 'Unspecified'
            ET.SubElement(ob, 'truncated').text = '0'
            ET.SubElement(ob, 'difficult').text = '0'
          
            bbox = ET.SubElement(ob, 'bndbox')
            ET.SubElement(bbox, 'xmin').text = str(int(x1))
            ET.SubElement(bbox, 'ymin').text = str(int(y1))
            ET.SubElement(bbox, 'xmax').text = str(int(x2))
            ET.SubElement(bbox, 'ymax').text = str(int(y2))
        
        f.close()

        tree = ET.ElementTree(annotation)
        tree.write(save_path)     
            
    except Exception as ex:
        print("Generate {0} failed, with error msg: {1}.".format(filename, ex.__str__()))

In [58]:
if __name__ == '__main__':
    #folder_path = r'C:\Users\Jake\Desktop\convert'
    folder_path = r'D:\Image\dataset\aaa'
    assert os.path.exists(folder_path), "Target directory is not exist: {}".format(folder_path) # 폴더 존재여부 확인
        
    for file in os.listdir(folder_path):
        if file.split('.')[1] == 'jpg':
            text_to_xml(folder_path, file)